### Manipulator Inverse Dynamics (ID)

In [ ]:
import numpy as np
np.set_printoptions(precision=2)
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
from mujoco_py import functions
from mujoco_parser import MuJoCoParserClass
from util import get_colors
print ("Done.")

### Simulate gravity compensation (spring-like behavior)

In [ ]:
env = MuJoCoParserClass(name='Panda',rel_path='../asset/panda/franka_panda.xml',VERBOSE=False)
env.init_viewer(TERMINATE_GLFW=True,window_width=0.5,window_height=0.75,cam_distance=3.0,cam_elevation=-45)
env.set_max_sec(max_sec=10.0)

# Set initial manipulator position 
q_rev_des = np.array([0,0,0,-90,0,90,0])*np.pi/180.0 # desired position
env.forward(q_rev=q_rev_des + 0.0*np.random.randn(env.n_rev_joint)*np.pi/180.0) # perturbation

# Buffers
sec_list = np.zeros(env.max_tick)
q_list   = np.zeros((env.max_tick,env.n_rev_joint))
while env.IS_ALIVE():    
    
    # Backup
    qpos_bu = env.sim.data.qpos[env.rev_joint_idxs].copy()
    qvel_bu = env.sim.data.qvel[env.rev_joint_idxs].copy()
    qacc_bu = env.sim.data.qacc[env.rev_joint_idxs].copy()
    
    # Emulate spring
    q_rev = env.get_q_rev()
    q_err = q_rev_des-q_rev
    env.sim.data.qpos[env.rev_joint_idxs] = q_rev
    env.sim.data.qvel[env.rev_joint_idxs] = np.zeros(env.n_rev_joint)
    env.sim.data.qacc[env.rev_joint_idxs] = np.zeros(env.n_rev_joint) # 20.0*q_err
    functions.mj_inverse(env.sim.model,env.sim.data)
    torque = env.sim.data.qfrc_inverse[env.rev_joint_idxs].copy()
    
    # Restore
    env.sim.data.qpos[env.rev_joint_idxs] = qpos_bu
    env.sim.data.qvel[env.rev_joint_idxs] = qvel_bu
    env.sim.data.qacc[env.rev_joint_idxs] = qacc_bu
    
    # Step
    env.step_and_render(torque=1.00*torque,render_speedup=1.0,RENDER_ALWAYS=False)
    env.print(print_every_sec=1.0,VERBOSE=1)
    # Append
    sec_list[env.tick-1] = env.get_sec_sim()
    q_list[env.tick-1,:] = env.get_q_rev()
    
print ("Done.")

In [ ]:
env.terminate_viewer() # terminate viewer

### Plot joint trajectories

In [ ]:
plt.figure(figsize=(10,5))
colors = get_colors(n=env.n_rev_joint,cm=plt.cm.Set1)
for i_idx in range(env.n_rev_joint):
    plt.plot(sec_list[1:],q_list[1:,i_idx],'-',color=colors[i_idx],lw=2,label='Joint %d'%(i_idx))
    plt.plot(sec_list[1:],q_rev_des[i_idx]+0.0*q_list[1:,i_idx],'--',color=colors[i_idx],
             label='Target %d'%(i_idx))
plt.legend(loc='lower right')
plt.title('Joint trajectories',fontsize=15)
plt.show()